In [1]:
!git clone https://github.com/ultralytics/yolov5 ../yolov5

fatal: целевой путь «../yolov5» уже существует и не является пустым каталогом.


In [1]:
import torch
import numpy as np
import os, sys
from PIL import Image
import cv2
import pandas as pd
import math
from torch import nn
from matplotlib import pyplot as plt
from skimage.feature import match_template
from torchvision import models
from torchvision import transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
sys.path.append('../yolov5/')

In [4]:
# yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/detect/yolo_finetuned_v1.pt')

In [5]:
yolo_model = torch.load('../models/detect/yolo_scratch_v1.pt', map_location=device)['model'].float().eval().autoshape()

Adding AutoShape... 


In [6]:
# yolo_model(np.random.randn(3, 640, 640)).pandas().xywh[0]

In [7]:
pd.read_csv('csv/results.csv', sep=',', index_col=0)

,filename,is_animal_there,is_it_a_dog,is_the_owner_there,color,tail,cam_id,address
0,B1.jpg,0,0,0,0,0,PVN_hd_SVAO_3498_4,"город Москва, улица Корнейчука, дом 41А"
1,B10.jpg,1,1,0,3,2,PVN_hd_UZAO_8772_3,"город Москва, улица Кадырова, дом 4"
2,B100.jpg,0,0,0,0,0,PVN_hd_SVAO_3055_1,"город Москва, проспект Мира, дом 182, корпус 3"
3,B101.jpg,1,0,0,0,0,PVN_hd_SVAO_1652_1,"город Москва, улица Академика Королёва, дом 28..."
4,B102.jpg,1,1,1,3,2,PVN_hd_UZAO_8772_3,"город Москва, улица Кадырова, дом 4"
...,...,...,...,...,...,...,...,...
309,С95.jpg,1,0,0,0,0,PVN_hd_SAO_3465_1,"город Москва, Клязьминская улица, дом 5, корпус 1"
310,С96.jpg,1,1,0,3,2,PVN_hd_TSAO_3647_1,"город Москва, Центросоюзный переулок, дом 4"
311,С97.jpg,1,1,1,3,2,PVN_hd_TSAO_5300_3,"город Москва, Верхняя Сыромятническая улица, д..."
312,С98.jpg,1,1,0,3,2,PVN_hd_TSAO_3647_1,"город Москва, Центросоюзный переулок, дом 4"


In [8]:
class MultiOutputModel(nn.Module):
    def __init__(self, n_color_classes=3, n_tail_classes=2):
        super().__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.base_model = nn.Sequential(
            *(list(self.resnet.children())[:-1])
        )  # take the model without classifier

        last_channel = (
            models.resnet34().fc.in_features
        )  # size of the layer before the classifier

        # create separate classifiers for our outputs
        self.color = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=last_channel, out_features=n_color_classes),
        )
        self.tail = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=last_channel, out_features=n_tail_classes),
        )

    def forward(self, x):
        x = self.base_model(x)

        # reshape from [batch, channels, 1, 1] to [batch, channels] to put it into classifier
        x = torch.flatten(x, start_dim=1)
        return {
            "color": self.color(x),
            "tail": self.tail(x),
        }

In [9]:
classifier_model = MultiOutputModel()

In [10]:
classifier_model = classifier_model.to(device)

In [11]:
weigths = torch.load('../models/classifier/resnet.pt', map_location=device)['model']

In [12]:
classifier_model.load_state_dict(weigths)

<All keys matched successfully>

In [13]:
tr_pipe = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)),
])

In [14]:
# tr_pipe(torch.from_numpy(np.random.rand(3, 640, 640).astype(np.uint8)))

In [15]:
images_folder = '../data/test_images/'

In [16]:
images = [os.path.join(images_folder, path) for path in os.listdir(images_folder)]
images = sorted(images)
images[:5]

['../data/test_images/B1.jpg',
 '../data/test_images/B10.jpg',
 '../data/test_images/B100.jpg',
 '../data/test_images/B101.jpg',
 '../data/test_images/B102.jpg']

In [17]:
all_dicts = []
for image in images:
    attr_dict = {'filename': image.split('/')[-1], 'is_animal_there': 0, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'tail': 0}
    cv_image = cv2.imread(image)
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    res_df = yolo_model(cv_image).pandas().xywh[0]
    animals = res_df.query("name=='dog' or name=='cat' or name=='bird'")
    if len(animals) > 0:
        attr_dict['is_animal_there'] = 1
        dogs = res_df.query("name=='dog'")
        if len(dogs) > 0:
            attr_dict['is_it_a_dog'] = 1
            best_dog = dogs.sort_values(by='confidence', ascending=False).iloc[0]
            coords = (int(best_dog['ycenter']-best_dog['height']//2),
                      int(best_dog['ycenter']+best_dog['height']//2), 
                      int(best_dog['xcenter']-best_dog['width']//2),
                      int(best_dog['xcenter']+best_dog['width']//2))
            dog_crop = cv_image[coords[0]:coords[1], coords[2]:coords[3]]
            classes = classifier_model(tr_pipe(dog_crop).unsqueeze(0).to(device))
            color_cl = classes['color'].argmax().cpu().detach().item() + 1
            tail_cl = classes['tail'].argmax().cpu().detach().item() + 1
            attr_dict['color'] = color_cl
            attr_dict['tail'] = tail_cl
            humans = res_df.query("name=='person'")
            if len(humans) > 0:
                best_dog_coords = best_dog['xcenter'] ,best_dog['ycenter']
                for i, r in humans.iterrows():
                    person_coords = r['xcenter'], r['ycenter']
                    dist = math.dist(person_coords, best_dog_coords)
                    if dist <= 300:
                        attr_dict['is_the_owner_there'] = 1
    all_dicts.append(attr_dict)

In [18]:
ai_results = pd.DataFrame(all_dicts)
ai_results.head(7)

,filename,is_animal_there,is_it_a_dog,is_the_owner_there,color,tail
0,B1.jpg,0,0,0,0,0
1,B10.jpg,1,1,0,3,2
2,B100.jpg,0,0,0,0,0
3,B101.jpg,1,0,0,0,0
4,B102.jpg,1,1,1,3,2
5,B103.jpg,1,1,0,3,1
6,B104.jpg,1,1,1,3,1


In [24]:
# ai_results.to_csv('test_results_5.csv')

In [25]:
addr_and_cams = pd.read_csv('csv/addr_and_cams.csv', sep=',', index_col=0)
addr_and_cams = addr_and_cams.sort_values(by='filename', ascending=True)
addr_and_cams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 8 to 310
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  314 non-null    object
 1   cam_id    240 non-null    object
 2   address   220 non-null    object
dtypes: object(3)
memory usage: 9.8+ KB


In [26]:
not_null = addr_and_cams[addr_and_cams.notna().all(axis=1)]
not_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 8 to 310
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  220 non-null    object
 1   cam_id    220 non-null    object
 2   address   220 non-null    object
dtypes: object(3)
memory usage: 6.9+ KB


In [27]:
with_null = addr_and_cams[addr_and_cams.isna().any(axis=1)]
with_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 309 to 303
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  94 non-null     object
 1   cam_id    20 non-null     object
 2   address   0 non-null      object
dtypes: object(3)
memory usage: 2.9+ KB


In [41]:
# template_im = cv2.imread('../data/test_images/B106.jpg')
# template_im = cv2.cvtColor(template_im, cv2.COLOR_BGR2GRAY)

In [42]:
# test_on_im = cv2.imread('../data/test_images/А28.jpg')
# test_on_im = cv2.cvtColor(test_on_im, cv2.COLOR_BGR2GRAY)

In [43]:
# template_im.shape, test_on_im.shape

In [44]:
# not_null_addrs[not_null_addrs['filename'] == 'B113.jpg']

In [45]:
# res = cv2.matchTemplate(test_on_im, template_im, cv2.TM_CCOEFF_NORMED)
# np.where(res[0] >= 0.5)[0].shape

In [46]:
# %matplotlib
# plt.figure(figsize = (20,20))
# plt.imshow(np.squeeze(nd_array))
# plt.show()

In [47]:
# for _, r1 in null_addrs.iterrows():
#     print(isinstance(r1['cam_id'], float))

In [ ]:
for _, r1 in with_null.iterrows():
    
    template_path = os.path.join(images_folder, r1['filename'])
    template_im = cv2.imread(template_path)
    template_im = cv2.cvtColor(template_im, cv2.COLOR_BGR2GRAY)
    
    best_test_on = (-1, None, None)
    
    for _, r2 in not_null.iterrows():
#         print(f"Testing {r2['filename']}, current tuple {best_test_on} ")
        try:
            test_on_path = os.path.join(images_folder, r2['filename'])
            test_on_im = cv2.imread(test_on_path)
            test_on_im = cv2.cvtColor(test_on_im, cv2.COLOR_BGR2GRAY)
            res = cv2.matchTemplate(test_on_im, template_im, cv2.TM_CCOEFF_NORMED)
            score = np.where(res[0] >= 0.5)[0].shape[0]
            if best_test_on[0] < score:
                best_test_on = (score, r2['address'], r2['cam_id'] if isinstance(r1['cam_id'], float) else r1['cam_id'])
            del test_on_im, res
        except:
            pass
    
    print(f"Best tuple for {r1['filename']} is {best_test_on}")
    
    addr_and_cams.loc[addr_and_cams['filename'] == r1['filename'], 'address'] = best_test_on[1]
    addr_and_cams.loc[addr_and_cams['filename'] == r1['filename'], 'cam_id'] = best_test_on[2]
    
#     print(addr_and_cams.loc[addr_and_cams['filename'] == r1['filename']])
    
    del template_im

Best tuple for B102.jpg is (618, 'город Москва, Клязьминская улица, дом 5, корпус 1', 'PVN_hd_SAO_3465_1')
Best tuple for B105.jpg is (260, 'город Москва, Верхняя Сыромятническая улица, дом 2', 'PVN_hd_TSAO_5300_3')
Best tuple for B106.jpg is (239, 'город Москва, Верхняя Сыромятническая улица, дом 2', 'PVN_hd_TSAO_5300_3')


In [49]:
addr_and_cams.to_csv('csv/addr_and_cams_fixed.csv')

In [35]:
# addr_and_cams = pd.read_csv('csv/addr_and_cams_fixed.csv', index_col=0).reset_index(drop=True)

In [36]:
results = pd.concat([ai_results, addr_and_cams[['cam_id', 'address']]], axis=1)
results.head(10)

,filename,is_animal_there,is_it_a_dog,is_the_owner_there,color,tail,cam_id,address
0,B1.jpg,0,0,0,0,0,PVN_hd_SVAO_1652_1,"город Москва, улица Академика Королёва, дом 28..."
1,B10.jpg,1,1,0,3,2,PVN_hd_SVAO_1652_1,"город Москва, улица Академика Королёва, дом 28..."
2,B100.jpg,0,0,0,0,0,PVN_hd_UZAO_8772_3,"город Москва, улица Кадырова, дом 4"
3,B101.jpg,1,0,0,0,0,PVN_hd_UZAO_8772_3,"город Москва, улица Кадырова, дом 4"
4,B102.jpg,1,1,1,3,2,PVN_hd_SAO_3465_1,"город Москва, Клязьминская улица, дом 5, корпус 1"
5,B103.jpg,1,1,0,3,1,PVN_hd_UZAO_8740_4,"город Москва, улица Академика Семёнова, дом 3"
6,B104.jpg,1,1,1,3,1,PVN_hd_SVAO_3055_1,"город Москва, проспект Мира, дом 182, корпус 3"
7,B105.jpg,1,0,0,0,0,PVN_hd_TSAO_5300_3,"город Москва, Верхняя Сыромятническая улица, д..."
8,B106.jpg,1,0,0,0,0,PVN_hd_TSAO_5300_3,"город Москва, Верхняя Сыромятническая улица, д..."
9,B107.jpg,1,1,0,3,1,UVN_SAO_027_7,"САО Ленинградский проспект, дом 36"


In [37]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   filename            314 non-null    object
 1   is_animal_there     314 non-null    int64 
 2   is_it_a_dog         314 non-null    int64 
 3   is_the_owner_there  314 non-null    int64 
 4   color               314 non-null    int64 
 5   tail                314 non-null    int64 
 6   cam_id              314 non-null    object
 7   address             314 non-null    object
dtypes: int64(5), object(3)
memory usage: 19.8+ KB


In [39]:
results.to_csv('csv/results.csv')